In [2]:
import openbabel
import pybel
import sys
import os
import os.path
import csv
import pandas as pd
import argparse
import numpy as np
import itertools
import shutil
from itertools import combinations
import re
import collections
from collections import OrderedDict
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
basedir = 'C:/Users/wellawat/Downloads/conf_energies'
datadir = 'C:/Users/wellawat/Downloads/conf_energies/complete_dataset'

### 1) Rename files to keep them consistent
```
for filename in os.listdir(datadir):
    splt = filename.split('_')
    print(filename)
    if splt[1] != 'm':
        new_ext = '_'.join(splt[1:])
        os.rename(f'{datadir}/{filename}', f'{datadir}/{splt[0]}_m_1_{new_ext}')
        #os.rename(filename, f'{}')
```

# Don't run 2,3,4 again. Start from 5!

### 2) Get SCF done energies and turn into a dataframe

In [4]:
def create_init_df(scf_file,outname ='scf_energies.csv', cols=['filename', 'energy']):
    data = pd.read_csv(scf_file, sep=' ', header=None, names=cols) 
    nrows = data.shape[0]
    for i in range(nrows):
        fname = data['filename'][i].split('/')[-1]
        data['filename'][i] = fname.split('.')[0]

    data.to_csv(outname,index=False)

In [5]:
create_init_df(f'{basedir}/complete_dataset_scf_done.txt',f'{basedir}/complete_dataset_scf_done.csv')

In [6]:
data_init = pd.read_csv(f'{basedir}/complete_dataset_scf_done.csv')
print(data_init.shape)
data_init.head()

(9454, 2)


,filename,energy
0,ABADIS01_m_1_gas_R,-838.509718
1,ABADIS01_m_1_gas_xR,-838.512341
2,ABADIS01_m_1_water_R,-838.537023
3,ABADIS01_m_1_water_xR,-838.537233
4,ABADIS_m_1_gas_R,-838.511238


In [7]:
## combine energies re-run for files with  NT but ended abruptly

df1 = pd.read_csv(f'{basedir}/complete_dataset_scf_done.csv')
df2 = pd.read_csv(f'{basedir}/highE_NT_rerun_scf_done.csv')
data_init = pd.concat([df1,df2])
print(data_init.shape)
data_init.to_csv(f'{basedir}/complete_dataset_wNTs_scf_done.csv',index=False)
data_init.head()

(9527, 2)


,filename,energy
0,ABADIS01_m_1_gas_R,-838.509718
1,ABADIS01_m_1_gas_xR,-838.512341
2,ABADIS01_m_1_water_R,-838.537023
3,ABADIS01_m_1_water_xR,-838.537233
4,ABADIS_m_1_gas_R,-838.511238


### 3) Find NAN files

In [8]:
nan_files = list(data_init.loc[data_init['energy'].isna()]['filename'])
print(len(nan_files))
with open(f'{basedir}/dataset_nan_list.txt','w') as f:
    for file in nan_files:
        f.write(f'{file}\n')
        
f.close()

270


In [ ]:
## move nan_files to a new folder
#for i,file in enumerate(nan_files[10:]):
#    shutil.move(f'{datadir}/{file}.out',f'{datadir}/nan_files/{file}.out')

### 4) Drop nan files

In [9]:
data_init = pd.read_csv(f'{basedir}/complete_dataset_wNTs_scf_done.csv')
data_init = data_init.dropna()
data_init = data_init.reset_index(drop=True)
data_init.to_csv(f'{basedir}/complete_dataset_final_scf_done.csv',index=False)
data_init.head()

,filename,energy
0,ABADIS01_m_1_gas_R,-838.509718
1,ABADIS01_m_1_gas_xR,-838.512341
2,ABADIS01_m_1_water_R,-838.537023
3,ABADIS01_m_1_water_xR,-838.537233
4,ABADIS_m_1_gas_R,-838.511238


# Start from here

In [10]:
data_init = pd.read_csv(f'{basedir}/complete_dataset_final_scf_done.csv')
print(data_init.shape)

(9257, 2)


### 5) Keep only max conf energy component when computing $\Delta E_{AB} = E_{A} - E_{B}$ 

In [11]:
data_init = pd.read_csv(f'{basedir}/complete_dataset_final_scf_done.csv')
print(data_init.shape)
grouped = np.array([list(g) for _, g in itertools.groupby(sorted(list(data_init['filename'])), lambda x: x[:6])])

(9257, 2)


In [12]:
#g_fname1 = [] g_fname2 = [] w_fname1 = [] w_fname2 = []

gas_del_confE = defaultdict(list)
wat_del_confE = defaultdict(list)
gas_m1m2  = defaultdict(list)
wat_m1m2  = defaultdict(list)

for g in range(len(grouped)):
    gas_c_files = []
    gas_c_energy = []
    wat_c_files = []
    wat_c_energy = []

    for i in itertools.combinations(grouped[g],2): 
        splt1 = i[0].split('_') 
        splt2 = i[1].split('_') 
        ref1 = splt1[0]
        ref2 = splt2[0]
        m1 = splt1[2]
        m2 = splt2[2]
        ph1,job1 = splt1[-2:]
        ph2,job2 = splt2[-2:]
    
        if ph1 == ph2:
            
            if job1==job2=='xR' and ref1 != ref2:
                conf_e1 =  data_init.loc[data_init['filename'] == i[0], 'energy'].iloc[0]
                conf_e2 =  data_init.loc[data_init['filename'] == i[1], 'energy'].iloc[0]
                delta_conf = abs(conf_e1-conf_e2)
                
                if ph1 == 'gas': 
                    gas_c_energy.append(delta_conf)
                    gas_c_files.append([i[0],i[1]])
                else: 
                    wat_c_energy.append(delta_conf)
                    wat_c_files.append([i[0],i[1]])
                    #print('appending to water', i[0],i[1])
    # when m1-m2 combinations are present. create a dictionary by refs and get max of mx-my pairs
    
    if len(gas_c_energy) != 0:
        for j in range(len(gas_c_energy)):
            ref1 = gas_c_files[j][0].split('_')[0]
            ref2 = gas_c_files[j][1].split('_')[0]
            m1 = gas_c_files[j][0].split('_')[2]
            m2 = gas_c_files[j][1].split('_')[2]
            
            key = f'{ref1}_{ref2}'
            gas_del_confE[key].append(gas_c_energy[j])
            gas_m1m2[key].append(f'{m1}_{m2}')
            
    if len(wat_c_energy) != 0:
        for j in range(len(wat_c_energy)):
            ref1 = wat_c_files[j][0].split('_')[0]
            ref2 = wat_c_files[j][1].split('_')[0]
            m1 = wat_c_files[j][0].split('_')[2]
            m2 = wat_c_files[j][1].split('_')[2]
            
            key = f'{ref1}_{ref2}'
            wat_del_confE[key].append(wat_c_energy[j])
            wat_m1m2[key].append(f'{m1}_{m2}')

## keep only the max conf E's per group
## drop  the A_m1 - A_m2 instances

gas_confE_dict = {}
gas_confM_dict = {}
for k,v in gas_del_confE.items():
    gas_confE_dict[k] = max(v)*2625.5
    gas_confM_dict[k] = gas_m1m2[k][np.argmax(v)]

wat_confE_dict = {}
wat_confM_dict = {}
for k,v in wat_del_confE.items():
    wat_confE_dict[k] = max(v)*2625.5
    wat_confM_dict[k] = wat_m1m2[k][np.argmax(v)]

In [13]:
## gas conformational energy dataframe
def save_conf_df(conf_dict,phase,MM_dict):
    df = pd.DataFrame.from_dict(conf_dict, orient='index',columns=['Delta_conformational_energy(kJ/mol)'])
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Refcodes'})
    df['components'] = list(MM_dict.values())
    df.to_csv(f'{basedir}/{phase}_dataset_coformational_energy.csv',index=False,header=True)
    df.head()

In [15]:
save_conf_df(gas_confE_dict,'Gas',gas_confM_dict)

In [16]:
save_conf_df(wat_confE_dict,'Water',wat_confM_dict)

In [17]:
df_gas_conf= pd.read_csv(f'{basedir}/Gas_dataset_coformational_energy.csv')
df_wat_conf = pd.read_csv(f'{basedir}/Water_dataset_coformational_energy.csv')

In [18]:
print(df_gas_conf.shape,df_wat_conf.shape)

(1302, 3) (1298, 3)


### 6) Compute $\Delta \Delta E_{strain}$ 

In [19]:
data_init = pd.read_csv(f'{basedir}/complete_dataset_final_scf_done.csv')
grouped = np.array([list(g) for _, g in itertools.groupby(sorted(list(data_init['filename'])), lambda x: x[:6])])
data_init.head()

,filename,energy
0,ABADIS01_m_1_gas_R,-838.509718
1,ABADIS01_m_1_gas_xR,-838.512341
2,ABADIS01_m_1_water_R,-838.537023
3,ABADIS01_m_1_water_xR,-838.537233
4,ABADIS_m_1_gas_R,-838.511238


In [20]:
gas_del_strainE = defaultdict(list)
wat_del_strainE = defaultdict(list)
gas_m1m2= defaultdict(list)
wat_m1m2 = defaultdict(list)

for g in range(len(grouped)):
    group = grouped[g]
    gas_s_files = []
    gas_s_energy = []
    wat_s_files = []
    wat_s_energy = []
    strain_group_gas = defaultdict(list)
    strain_group_wat = defaultdict(list)
    
    for i in itertools.combinations(group,2): 
        splt1 = i[0].split('_') 
        splt2 = i[1].split('_') 
        ref1 = splt1[0]
        ref2 = splt2[0]
        m1 = splt1[2]
        m2 = splt2[2]
        ph1,job1 = splt1[-2:]
        ph2,job2 = splt2[-2:]
        
        if (ref1==ref2) and (m1==m2): # make sure to use only same ref and same component
            if ph1 == ph2 and job1 != job2:
            
                strain_e1 =  data_init.loc[data_init['filename'] == i[0], 'energy'].iloc[0]
                strain_e2 =  data_init.loc[data_init['filename'] == i[1], 'energy'].iloc[0]
                delta_strain = abs(strain_e1-strain_e2)

                if ph1 == 'gas': 
                    gas_s_energy.append(delta_strain)
                    gas_s_files.append([i[0],i[1]])
                else: 
                    wat_s_energy.append(delta_strain)
                    wat_s_files.append([i[0],i[1]])
                
                        
    if len(gas_s_energy) != 0:
        for j in range(len(gas_s_energy)):
            ref1 = gas_s_files[j][0].split('_')[0]
            ref2 = gas_s_files[j][1].split('_')[0]
            m1 = gas_s_files[j][0].split('_')[2]
            
            key = f'{ref1}'
            gas_del_strainE[key].append(gas_s_energy[j])
            gas_m1m2[key].append(f'{m1}')
            
    if len(wat_s_energy) != 0:
        for j in range(len(wat_s_energy)):
            ref1 = wat_s_files[j][0].split('_')[0]
            ref2 = wat_s_files[j][1].split('_')[0]
            m1 = wat_s_files[j][0].split('_')[2]
            key = f'{ref1}'
            wat_del_strainE[key].append(wat_s_energy[j])
            wat_m1m2[key].append(f'{m1}')
            
## find max per refcode
gas_strainE = {}
gas_strainM = {}

for k,v in gas_del_strainE.items():
    gas_strainE[k] = max(v)*2625.5
    gas_strainM[k] = gas_m1m2[k][np.argmax(v)]
    #keep_id = np.argmax(v)

wat_strainE = {}
wat_strainM = {}
for k,v in wat_del_strainE.items():
    wat_strainE[k] = max(v)*2625.5
    wat_strainM[k] = wat_m1m2[k][np.argmax(v)]

In [21]:
def finf_DD_strain(strain_dict,phase,MM_dict):
    dd_strainE = defaultdict(list)
    mms = []
    keys = list(strain_dict.keys())
    grouped = np.array([list(g) for _, g in itertools.groupby(sorted(list(keys)), lambda x: x[:6])])
    
    for g in grouped:
        if len(g)>1:
            for pair in itertools.combinations(g,2): 
                ref1 = pair[0]
                ref2 = pair[1]
                m1 = MM_dict[ref1]
                m2 = MM_dict[ref2]
                ddS = abs(strain_dict[ref1] - strain_dict[ref2])
                dd_strainE[f'{ref1}_{ref2}'].append(ddS)
                mms.append(f'{m1}_{m2}')
    
    df = pd.DataFrame.from_dict(dd_strainE, orient='index',columns=['Delta_strain_energy'])
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Refcodes'})
    df['components'] = mms
    df.to_csv(f'{basedir}/{phase}_dataset_strain_energy.csv',index=False,header=True)

In [22]:
finf_DD_strain(gas_strainE,'Gas',gas_strainM)

In [23]:
finf_DD_strain(wat_strainE,'Water',wat_strainM)

In [24]:
df_gas_S = pd.read_csv(f'{basedir}/Gas_dataset_strain_energy.csv')
df_wat_S = pd.read_csv(f'{basedir}/Water_dataset_strain_energy.csv')

In [ ]:
df_gas_S = df_gas_S.loc[(df_gas_S!=0).any(axis=1)]
df_wat_S = df_wat_S.loc[(df_wat_S!=0).any(axis=1)]

## Plot energies

In [ ]:
df_gas_C= pd.read_csv(f'{basedir}/Gas_coformational_energy.csv')
df_wat_C = pd.read_csv(f'{basedir}/Water_coformational_energy.csv')
df_gas_S = pd.read_csv(f'{basedir}/Gas_strain_energy.csv')
df_wat_S = pd.read_csv(f'{basedir}/Water_strain_energy.csv')

In [ ]:
energies = [df_gas_C,df_wat_C,df_gas_S,df_wat_S]
phases = ['gas','water','gas','water']
out_type = ['conf','conf','strain','strain']
types = ['Delta_conformational_energy(kJ/mol)','Delta_conformational_energy(kJ/mol)','Delta_strain_energy','Delta_strain_energy']
xlabels = [r'$\Delta$ Conformation energy (kJ/mol)',r'$\Delta$ Conformation energy (kJ/mol)',r'$\Delta$ Strain energy (kJ/mol)',r'$\Delta$ Strain energy (kJ/mol)']

In [ ]:
def plot_histogram(elist,outname,xlabel,density=True):
    fig, ax = plt.subplots(figsize=(8,6))
    n, bins, patches = ax.hist(elist,100,density=True, range=(0.00,200))
    #csum = np.cumsum(n)
    #ax.plot(bins[1:],n)
    #ax.invert_xaxis()
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Probability density')
    ax.set_title('Conformational energy distribution')
    fig.savefig(outname, dpi=600, facecolor='w')
    fig.show()

In [ ]:
df_gas_S = df_gas_S.sort_values(by=['Delta_strain_energy'])


In [ ]:
csum = df_gas_S.cumsum()
csum['Delta_strain_energy']

In [ ]:
sums = list(csum['Delta_strain_energy'])
tot = np.sum(sums)
plt.plot(np.arange(len(sums)),sums/tot)

In [ ]:
c_energies = list(df_gas_S['Delta_strain_energy'])
type(c_energies[2])

In [ ]:
c_energies = list(df_gas_S['Delta_strain_energy'])
energies= []
for c in c_energies:
    if c < 500.0:
        energies.append(c)

In [ ]:
plot_histogram(energies,'xx',density=True)

In [ ]:

for i in range(len(grouped)):
    group = grouped[i]
    gas_free_files = []
    gas_free_energy = []
    wat_free_files = []
    wat_free_energy = []
    for i in itertools.combinations(group,2): 
        
        splt = filename.split('_')
        ref = splt[0]
        idx = splt[2] 
        phase = splt[-2]
        opt = splt[-1]

for i in itertools.combinations(grouped[2], 2):
    if i[0].split('_')[-2] == i[1].split('_')[-2]:
        print(i)

## OLD CODES

In [ ]:
''' #if there's more than one delta conf Es per group per solvent this won't work 
    if len(gas_c_energy) > 1:
        max_id = np.argmax(gas_c_energy)
        gas_del_confE.append(np.max(gas_c_energy))
        ref1 = gas_c_files[max_id][0].split('_')[0]
        ref2 = gas_c_files[max_id][1].split('_')[0]
        m1 = gas_c_files[max_id][0].split('_')[2]
        m2 = gas_c_files[max_id][1].split('_')[2]
        g_fname1.append(f'{ref1}_{ref2}')
        g_fname2.append(f'm{m1}_m{m2}')
      
        
    if len(wat_c_energy) > 1:
        
        max_id = np.argmax(wat_c_energy)
        wat_del_confE.append(np.max(wat_c_energy))
        ref1 = wat_c_files[max_id][0].split('_')[0]
        ref2 = wat_c_files[max_id][1].split('_')[0]
        m1 = wat_c_files[max_id][0].split('_')[2]
        m2 = wat_c_files[max_id][1].split('_')[2]
        w_fname1.append(f'{ref1}_{ref2}')
        w_fname2.append(f'm{m1}_m{m2}')
        
    if len(gas_c_energy) == 1:
        gas_del_confE.append(gas_c_energy)
        ref1 = gas_c_files[0][0].split('_')[0]
        ref2 = gas_c_files[0][1].split('_')[0]
        m1 = gas_c_files[0][0].split('_')[2]
        m2 = gas_c_files[0][1].split('_')[2]
        g_fname1.append(f'{ref1}_{ref2}')
        g_fname2.append(f'm{m1}_m{m2}')
        
    if len(wat_c_energy) == 1:
        wat_del_confE.append(wat_c_energy)
        ref1 = wat_c_files[0][0].split('_')[0]
        ref2 = wat_c_files[0][1].split('_')[0]
        m1 = wat_c_files[0][0].split('_')[2]
        m2 = wat_c_files[0][1].split('_')[2]
        w_fname1.append(f'{ref1}_{ref2}')
        w_fname2.append(f'm{m1}_m{m2}')
        '''

In [ ]:
import openbabel

In [ ]:
!babel complete_dataset/ABADIS_m_1_water_R.out ABADIS_m_1_water_R.mol2